In [ ]:
## Code for the publication 

## Validating and Utilizing Machine Learning Methods to Investigate 
## the Impacts of Synthesis Parameters in Gold Nanoparticle Synthesis"
## by Daniel Schletz, Morten Breidung, and Andreas Fery

## CC BY-NC-ND 4.0

In [1]:
import pandas as pd
import numpy as np
import os
import matplotlib.pyplot as plt
import xgboost as xgb
import pickle
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import AdaBoostRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.multioutput import MultiOutputRegressor
from sklearn.tree import DecisionTreeRegressor
from xgboost import XGBRegressor
from sklearn.preprocessing import MaxAbsScaler
from numpy import mean
from numpy import std
from sklearn.model_selection import KFold
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import r2_score
from sklearn.metrics import mean_squared_error

In [2]:
#Import Parameter+DLS
a = pd.read_excel("Data/Synthesis_Parameters_DLS.xlsx")
#Import Spectra
b = pd.read_excel("Data/UV-vis_Spectra.xlsx")
#Import Abs_Max Data
c = pd.read_excel("Data/Absorption_Maximum_FW80M.xlsx")

In [4]:
#Prepare Parameters
X = a.drop(columns=["Name","DLS Diameter [nm]","Standard Deviation DLS [nm]"])

In [5]:
#### Begin Data Collection

In [6]:
def NestedCV_LR(XData, yData, savetest, savehat):
    # configure the cross-validation procedure
    cv_outer = KFold(n_splits=10, shuffle=True, random_state=42)
    # enumerate splits
    outer_results = list()
    outer_results2 = list()
    y_graph_hat = list()
    y_graph_test = list()
    X2 = XData.to_numpy()
    y2 = yData.to_numpy().ravel()
    for train_ix, test_ix in cv_outer.split(X):
    # split data
        X_train, X_test = X2[train_ix, :], X2[test_ix, :]
        y_train, y_test = y2[train_ix], y2[test_ix]
        y_graph_test.append(y_test)
        # configure the cross-validation procedure
        cv_inner = KFold(n_splits=3, shuffle=True, random_state=1)
        # define the model
        model = LinearRegression()
        # define search
        space = dict()
        search = GridSearchCV(model, space, scoring='r2', cv=cv_inner, refit=True, n_jobs=-1)
        # execute search
        result = search.fit(X_train, y_train)
        # get the best performing model fit on the whole training set
        best_model = result.best_estimator_
        # evaluate model on the hold out dataset
        yhat = best_model.predict(X_test)
        y_graph_hat.append(yhat)
        # evaluate the model
        acc = mean_squared_error(y_test, yhat)
        acc2 = r2_score(y_test, yhat)
        # store the result
        outer_results.append(acc)
        outer_results2.append(acc2)
        # report progress
        print('>acc[MSE]=%.3f, acc=%.3f, est=%.3f, cfg=%s' % (acc, acc2, result.best_score_, result.best_params_))
    # summarize the estimated performance of the model
    print('Accuracy: %.3f (%.3f)' % (mean(outer_results), std(outer_results)))
    print('Accuracy: %.3f (%.3f)' % (mean(outer_results2), std(outer_results2)))
    # save data
    f = open(savetest, 'wb')
    pickle.dump(y_graph_test, f)
    f.close()
    f = open(savehat, 'wb')
    pickle.dump(y_graph_hat, f)
    f.close()
    
    return y_graph_test, y_graph_hat

In [7]:
def NestedCV_RF(XData, yData, savetest, savehat):
    # configure the cross-validation procedure
    cv_outer = KFold(n_splits=10, shuffle=True, random_state=42)
    # enumerate splits
    outer_results = list()
    outer_results2 = list()
    y_graph_hat = list()
    y_graph_test = list()
    X2 = XData.to_numpy()
    y2 = yData.to_numpy().ravel()
    for train_ix, test_ix in cv_outer.split(X):
    # split data
        X_train, X_test = X2[train_ix, :], X2[test_ix, :]
        y_train, y_test = y2[train_ix], y2[test_ix]
        y_graph_test.append(y_test)
        # configure the cross-validation procedure
        cv_inner = KFold(n_splits=3, shuffle=True, random_state=42)
        # define the model
        model = RandomForestRegressor(random_state=42, n_jobs = -1)
        # define search space
        space = dict()
        space['n_estimators'] = [500, 700, 900, 1000, 1100, 1300, 1500]
        space['max_features'] = [1.0, 'sqrt', 0.3]
        space['min_samples_leaf'] = [1, 2, 4]
        space['min_samples_split'] = [2, 4, 8, 16]
        # define search
        search = GridSearchCV(model, space, scoring='r2', cv=cv_inner, refit=True, n_jobs=-1)
        # execute search
        result = search.fit(X_train, y_train)
        # get the best performing model fit on the whole training set
        best_model = result.best_estimator_
        # evaluate model on the hold out dataset
        yhat = best_model.predict(X_test)
        y_graph_hat.append(yhat)
        # evaluate the model
        acc = mean_squared_error(y_test, yhat)
        acc2 = r2_score(y_test, yhat)
        # store the result
        outer_results.append(acc)
        outer_results2.append(acc2)
        # report progress
        print('>acc[MSE]=%.3f, acc=%.3f, est=%.3f, cfg=%s' % (acc, acc2, result.best_score_, result.best_params_))
    # summarize the estimated performance of the model
    print('Accuracy: %.3f (%.3f)' % (mean(outer_results), std(outer_results)))
    print('Accuracy: %.3f (%.3f)' % (mean(outer_results2), std(outer_results2)))
    # save data
    f = open(savetest, 'wb')
    pickle.dump(y_graph_test, f)
    f.close()
    f = open(savehat, 'wb')
    pickle.dump(y_graph_hat, f)
    f.close()
    
    return y_graph_test, y_graph_hat

In [8]:
def NestedCV_AdaB(XData, yData, savetest, savehat):
    # configure the cross-validation procedure
    cv_outer = KFold(n_splits=10, shuffle=True, random_state=42)
    # enumerate splits
    outer_results = list()
    outer_results2 = list()
    y_graph_hat = list()
    y_graph_test = list()
    X2 = XData.to_numpy()
    y2 = yData.to_numpy().ravel()
    for train_ix, test_ix in cv_outer.split(X):
    # split data
        X_train, X_test = X2[train_ix, :], X2[test_ix, :]
        y_train, y_test = y2[train_ix], y2[test_ix]
        y_graph_test.append(y_test)
        # configure the cross-validation procedure
        cv_inner = KFold(n_splits=3, shuffle=True, random_state=1)
        # define the model
        model = AdaBoostRegressor(base_estimator = DecisionTreeRegressor(),random_state=42)
        # define search space
        space = dict()
        space['n_estimators'] = [300, 500]#, 700, 900, 1000, 1100, 1300, 1500]
        space['base_estimator__max_depth'] = [None, 6, 8, 10, 12] 
        space['learning_rate'] = [0.1, 0.25, 0.5, 0.75, 1.0, 1.25, 1.5]
        # define search
        search = GridSearchCV(model, space, scoring='r2', cv=cv_inner, refit=True, n_jobs=-1)
        # execute search
        result = search.fit(X_train, y_train)
        # get the best performing model fit on the whole training set
        best_model = result.best_estimator_
        # evaluate model on the hold out dataset
        yhat = best_model.predict(X_test)
        y_graph_hat.append(yhat)
        # evaluate the model
        acc = mean_squared_error(y_test, yhat)
        acc2 = r2_score(y_test, yhat)
        # store the result
        outer_results.append(acc)
        outer_results2.append(acc2)
        # report progress
        #print('>acc[MSE]=%.3f, est=%.3f, cfg=%s' % (acc, result.best_score_, result.best_params_))
        print('>acc[MSE]=%.3f, acc=%.3f, est=%.3f, cfg=%s' % (acc, acc2, result.best_score_, result.best_params_))
    # summarize the estimated performance of the model
    print('Accuracy: %.3f (%.3f)' % (mean(outer_results), std(outer_results)))
    print('Accuracy: %.3f (%.3f)' % (mean(outer_results2), std(outer_results2)))
    # save data
    f = open(savetest, 'wb')
    pickle.dump(y_graph_test, f)
    f.close()
    f = open(savehat, 'wb')
    pickle.dump(y_graph_hat, f)
    f.close()

    return y_graph_test, y_graph_hat

In [9]:
def NestedCV_XGB(XData, yData, savetest, savehat):
    # configure the cross-validation procedure
    cv_outer = KFold(n_splits=10, shuffle=True, random_state=42)
    # enumerate splits
    outer_results = list()
    outer_results2 = list()
    y_graph_hat = list()
    y_graph_test = list()
    X2 = XData.to_numpy()
    y2 = yData.to_numpy().ravel()
    for train_ix, test_ix in cv_outer.split(X):
    # split data
        X_train, X_test = X2[train_ix, :], X2[test_ix, :]
        y_train, y_test = y2[train_ix], y2[test_ix]
        y_graph_test.append(y_test)
        # configure the cross-validation procedure
        cv_inner = KFold(n_splits=3, shuffle=True, random_state=42)
        # define the model
        model = XGBRegressor(random_state=42, tree_method = 'exact', n_jobs = -1)
        # define search space
        space = dict()
        space['learning_rate'] = [0.1, 0.2, 0.3, 0.4, 0.5]
        space['max_depth']= [6]#None, 6]
        space['colsample_bytree'] = [0.5,1.0]
        space['colsample_bylevel'] = [0.5,1.0]
        # define search
        search = GridSearchCV(model, space, scoring='r2', cv=cv_inner, refit=True, n_jobs=-1)
        # execute search
        result = search.fit(X_train, y_train)
        # get the best performing model fit on the whole training set
        best_model = result.best_estimator_
        # evaluate model on the hold out dataset
        yhat = best_model.predict(X_test)
        y_graph_hat.append(yhat)
        # evaluate the model
        acc = mean_squared_error(y_test, yhat)
        acc2 = r2_score(y_test, yhat)
        # store the result
        outer_results.append(acc)
        outer_results2.append(acc2)
        # report progress
        print('>acc[MSE]=%.3f, acc=%.3f, est=%.3f, cfg=%s' % (acc, acc2, result.best_score_, result.best_params_))
    # summarize the estimated performance of the model
    print('Accuracy: %.3f (%.3f)' % (mean(outer_results), std(outer_results)))
    print('Accuracy: %.3f (%.3f)' % (mean(outer_results2), std(outer_results2)))
    #save data
    f = open(savetest, 'wb')
    pickle.dump(y_graph_test, f)
    f.close()
    f = open(savehat, 'wb')
    pickle.dump(y_graph_hat, f)
    f.close()

    return y_graph_test, y_graph_hat

In [10]:
#Prepare Abs_Max Target
y = c.drop(columns=["Name", "Abs max", "FW80M"])
test_MaxAbs_LR, hat_MaxAbs_LR = NestedCV_LR(X,y, "FW_LR_test.pckl", "FW_LR_hat.pckl")
test_MaxAbs_RF, hat_MaxAbs_RF = NestedCV_RF(X,y, "MaxAbs_RF_test.pckl", "MaxAbs_RF_hat.pckl")
test_MaxAbs_AdaB, hat_MaxAbs_AdaB = NestedCV_AdaB(X,y, "MaxAbs_AdaB_test.pckl", "MaxAbs_AdaB_hat.pckl")
test_MaxAbs_XGB, hat_MaxAbs_XGB = NestedCV_XGB(X,y, "MaxAbs_XGB_test.pckl", "MaxAbs_XGB_hat.pckl")

>acc[MSE]=375.941, acc=0.579, est=-3.546, cfg={}
>acc[MSE]=505.761, acc=0.385, est=0.036, cfg={}
>acc[MSE]=6447.830, acc=-38.284, est=-0.006, cfg={}
>acc[MSE]=850.320, acc=0.163, est=-0.084, cfg={}
>acc[MSE]=206.338, acc=0.392, est=-5.564, cfg={}
>acc[MSE]=424.217, acc=-0.406, est=-0.322, cfg={}
>acc[MSE]=153.523, acc=0.524, est=-0.693, cfg={}
>acc[MSE]=481.905, acc=0.144, est=-0.293, cfg={}
>acc[MSE]=143.292, acc=0.568, est=-1.569, cfg={}
>acc[MSE]=166.776, acc=0.203, est=-0.897, cfg={}
Accuracy: 975.590 (1835.854)
Accuracy: -3.573 (11.574)
>acc[MSE]=554.421, acc=0.379, est=0.540, cfg={'min_samples_leaf': 1}
>acc[MSE]=283.031, acc=0.656, est=0.482, cfg={'min_samples_leaf': 4}
>acc[MSE]=103.211, acc=0.371, est=0.546, cfg={'min_samples_leaf': 2}
>acc[MSE]=343.222, acc=0.662, est=0.444, cfg={'min_samples_leaf': 2}
>acc[MSE]=198.014, acc=0.417, est=0.480, cfg={'min_samples_leaf': 2}
>acc[MSE]=291.516, acc=0.034, est=0.589, cfg={'min_samples_leaf': 2}
>acc[MSE]=73.077, acc=0.773, est=0.516

In [11]:
#Prepare FW80M Target
y = c.drop(columns=["Name", "Abs max", "Wavelength [nm]"])
test_FW_LR, hat_FW_LR = NestedCV_LR(X,y, "FW_LR_test.pckl", "FW_LR_hat.pckl")
test_FW_RF, hat_FW_RF = NestedCV_RF(X,y, "FW_RF_test.pckl", "FW_RF_hat.pckl")
test_FW_AdaB, hat_FW_AdaB = NestedCV_AdaB(X,y, "FW_AdaB_test.pckl", "FW_AdaB_hat.pckl")
test_FW_XGB, hat_FW_XGB = NestedCV_XGB(X,y, "FW_XGB_test.pckl", "FW_XGB_hat.pckl")


>acc[MSE]=301.390, acc=0.576, est=-1.385, cfg={}
>acc[MSE]=413.010, acc=0.329, est=0.189, cfg={}
>acc[MSE]=2510.776, acc=-41.370, est=-0.470, cfg={}
>acc[MSE]=535.470, acc=-0.029, est=0.021, cfg={}
>acc[MSE]=98.533, acc=0.298, est=-11.777, cfg={}
>acc[MSE]=301.017, acc=-0.872, est=-0.274, cfg={}
>acc[MSE]=138.861, acc=-0.018, est=-0.136, cfg={}
>acc[MSE]=506.122, acc=-0.186, est=-0.397, cfg={}
>acc[MSE]=100.123, acc=0.568, est=-0.769, cfg={}
>acc[MSE]=75.021, acc=-0.789, est=-0.128, cfg={}
Accuracy: 498.032 (690.159)
Accuracy: -4.149 (12.416)
>acc[MSE]=478.956, acc=0.327, est=0.357, cfg={'min_samples_leaf': 4}
>acc[MSE]=292.845, acc=0.524, est=0.351, cfg={'min_samples_leaf': 4}
>acc[MSE]=60.394, acc=-0.019, est=0.316, cfg={'min_samples_leaf': 1}
>acc[MSE]=164.160, acc=0.685, est=0.140, cfg={'min_samples_leaf': 4}
>acc[MSE]=126.650, acc=0.098, est=0.425, cfg={'min_samples_leaf': 2}
>acc[MSE]=163.164, acc=-0.015, est=0.482, cfg={'min_samples_leaf': 2}
>acc[MSE]=64.544, acc=0.527, est=0.4

In [12]:
#Prepare DLS Target
y = a["DLS Diameter [nm]"]
test_Dia_LR, hat_Dia_LR = NestedCV_LR(X,y, "Dia_LR_test.pckl", "Dia_LR_hat.pckl")
test_Dia_RF, hat_Dia_RF = NestedCV_RF(X,y, "Dia_RF_test.pckl", "Dia_RF_hat.pckl")
test_Dia_AdaB, hat_Dia_AdaB = NestedCV_AdaB(X,y, "Dia_AdaB_test.pckl", "Dia_AdaB_hat.pckl")
test_Dia_XGB, hat_Dia_XGB = NestedCV_XGB(X,y, "Dia_XGB_test.pckl", "Dia_XGB_hat.pckl")


>acc[MSE]=415.958, acc=0.072, est=-0.159, cfg={}
>acc[MSE]=384.832, acc=0.132, est=-1.117, cfg={}
>acc[MSE]=871.284, acc=-0.343, est=-0.536, cfg={}
>acc[MSE]=488.280, acc=-0.061, est=-1.019, cfg={}
>acc[MSE]=148.638, acc=0.296, est=-1.080, cfg={}
>acc[MSE]=490.333, acc=-0.108, est=-1.400, cfg={}
>acc[MSE]=401.561, acc=0.094, est=-0.233, cfg={}
>acc[MSE]=386.812, acc=-2.071, est=-0.040, cfg={}
>acc[MSE]=257.915, acc=-0.367, est=-0.189, cfg={}
>acc[MSE]=662.736, acc=-0.417, est=-0.153, cfg={}
Accuracy: 450.835 (191.124)
Accuracy: -0.277 (0.639)
>acc[MSE]=288.782, acc=0.356, est=0.305, cfg={'min_samples_leaf': 2}
>acc[MSE]=323.674, acc=0.270, est=0.211, cfg={'min_samples_leaf': 4}
>acc[MSE]=407.552, acc=0.372, est=0.166, cfg={'min_samples_leaf': 1}
>acc[MSE]=371.954, acc=0.192, est=0.162, cfg={'min_samples_leaf': 2}
>acc[MSE]=121.708, acc=0.424, est=0.194, cfg={'min_samples_leaf': 1}
>acc[MSE]=262.253, acc=0.407, est=0.191, cfg={'min_samples_leaf': 2}
>acc[MSE]=186.383, acc=0.579, est=0.1

In [13]:
## Random Forest Full Spectrum
# configure the cross-validation procedure
cv_outer = KFold(n_splits=10, shuffle=True, random_state=42)
# enumerate splits
outer_results = list()
outer_results2 = list()
y_graph_hat_CombRF = list()
y_graph_test_CombRF = list()
#Normalize Spectra
scaler=MaxAbsScaler()
b = b.drop(columns=["Wavelength [nm]"])
y = pd.DataFrame(scaler.fit_transform(b.T).T,columns=b.columns)
X2 = X.to_numpy()
y2 = y.to_numpy()
for train_ix, test_ix in cv_outer.split(X):
# split data
    X_train, X_test = X2[train_ix, :], X2[test_ix, :]
    y_train, y_test = y2[train_ix], y2[test_ix]
    y_graph_test_CombRF.append(y_test)
    # configure the cross-validation procedure
    cv_inner = KFold(n_splits=3, shuffle=True, random_state=42)
    # define the model
    model = RandomForestRegressor(random_state=42, n_jobs = -1)
    # define search space
    space = dict()
    space['n_estimators'] = [500, 700, 900, 1000, 1100, 1300, 1500]
    space['max_features'] = [1.0, 'sqrt', 0.3]
    space['min_samples_leaf'] = [1, 2, 4]
    space['min_samples_split'] = [2, 4, 8, 16]
    # define search
    search = GridSearchCV(model, space, scoring='r2', cv=cv_inner, refit=True, n_jobs=-1)
    # execute search
    result = search.fit(X_train, y_train)
    # get the best performing model fit on the whole training set
    best_model = result.best_estimator_
    # evaluate model on the hold out dataset
    yhat = best_model.predict(X_test)
    y_graph_hat_CombRF.append(yhat)
    # evaluate the model
    acc = mean_squared_error(y_test, yhat)
    acc2 = r2_score(y_test, yhat)
    # store the result
    outer_results.append(acc)
    outer_results2.append(acc2)
    # report progress
    print('>acc=%.3f, est=%.3f, cfg=%s' % (acc, result.best_score_, result.best_params_))
    print('>acc=%.3f, est=%.3f, cfg=%s' % (acc2, result.best_score_, result.best_params_))
# summarize the estimated performance of the model
print('Accuracy: %.3f (%.3f)' % (mean(outer_results), std(outer_results)))
print('Accuracy: %.3f (%.3f)' % (mean(outer_results2), std(outer_results2)))

>acc=0.007, est=0.463, cfg={'min_samples_leaf': 2}
>acc=0.408, est=0.463, cfg={'min_samples_leaf': 2}
>acc=0.002, est=0.413, cfg={'min_samples_leaf': 2}
>acc=0.465, est=0.413, cfg={'min_samples_leaf': 2}
>acc=0.001, est=0.410, cfg={'min_samples_leaf': 1}
>acc=0.770, est=0.410, cfg={'min_samples_leaf': 1}
>acc=0.003, est=0.349, cfg={'min_samples_leaf': 2}
>acc=0.461, est=0.349, cfg={'min_samples_leaf': 2}
>acc=0.003, est=0.455, cfg={'min_samples_leaf': 2}
>acc=0.011, est=0.455, cfg={'min_samples_leaf': 2}
>acc=0.003, est=0.483, cfg={'min_samples_leaf': 2}
>acc=0.045, est=0.483, cfg={'min_samples_leaf': 2}
>acc=0.003, est=0.488, cfg={'min_samples_leaf': 2}
>acc=-0.818, est=0.488, cfg={'min_samples_leaf': 2}
>acc=0.004, est=0.483, cfg={'min_samples_leaf': 4}
>acc=0.217, est=0.483, cfg={'min_samples_leaf': 4}
>acc=0.004, est=0.281, cfg={'min_samples_leaf': 4}
>acc=0.457, est=0.281, cfg={'min_samples_leaf': 4}
>acc=0.001, est=0.323, cfg={'min_samples_leaf': 4}
>acc=0.661, est=0.323, cfg={'m

In [15]:
##Final model training for Random Forest Abs_Max for SHAP Analysis
y = c.drop(columns=["Name", "Abs max", "FW80M"])
cv_inner = KFold(n_splits=10, shuffle=True, random_state=1)
# define the model
model = RandomForestRegressor(random_state=42, n_jobs = -1)
space = dict()
space['n_estimators'] = [500, 700, 900, 1000, 1100, 1300, 1500]
space['max_features'] = [1.0, 'sqrt', 0.3]
space['min_samples_leaf'] = [1, 2, 4]
space['min_samples_split'] = [2, 4, 8, 16]
# define search
search = GridSearchCV(model, space, scoring='r2', cv=cv_inner, refit=True, n_jobs=-1)
# execute search
result = search.fit(X.to_numpy(), y.to_numpy().ravel())
print('>acc=%.3f, cfg=%s' % (result.best_score_, result.best_params_))


>acc=0.609, cfg={'max_features': 0.3, 'min_samples_leaf': 1, 'min_samples_split': 4, 'n_estimators': 1100}


In [17]:
model = search.best_estimator_
y = c.drop(columns=["Name", "Abs max", "FW80M"])
model.fit(X, y.to_numpy().ravel())

RandomForestRegressor(max_features=0.3, min_samples_split=4, n_estimators=1100,
                      n_jobs=-1, random_state=42)